In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install pandas-profiling

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd


In [17]:
# Load your dataset
data = pd.read_csv('C:/Users/United/downloads/hmmmm/chbmit_preprocessed_data.csv')

# Specify the desired size of the trimmed dataset
desired_size = 5000  # For example, reduce the dataset to 500000 data points

# Randomly sample the dataset
trimmed_data = data.sample(n=desired_size, random_state=42)

# Save the trimmed dataset
trimmed_data.to_csv('trimmed_dataset3.csv', index=False)

In [18]:
df = pd.read_csv('C:/Users/United/downloads/hmmmm/trimmed_dataset3.csv')


In [19]:
print(df)

      # FP1-F7     C3-P3     C4-P4         CZ-PZ     F3-C3     F4-C4  \
0    -0.000019  0.000056  0.000052  5.450000e-05  0.000115  0.000142   
1     0.000015  0.000021 -0.000041 -1.470000e-05 -0.000076 -0.000044   
2     0.000005  0.000006 -0.000029  2.930000e-06  0.000024  0.000029   
3    -0.000061 -0.000014 -0.000110 -9.350000e-05  0.000045  0.000075   
4    -0.000039 -0.000016 -0.000019 -1.310000e-05 -0.000015 -0.000022   
...        ...       ...       ...           ...       ...       ...   
4995 -0.000062 -0.000010 -0.000003 -5.860000e-07  0.000020 -0.000014   
4996  0.000186 -0.000073  0.000074  6.390000e-05 -0.000033  0.000064   
4997 -0.000136  0.000068  0.000050  1.500000e-05  0.000053  0.000029   
4998  0.000115 -0.000272  0.000208  6.650000e-05  0.000272 -0.000234   
4999  0.000012  0.000043  0.000013  4.160000e-05 -0.000006  0.000026   

         F7-T7     F8-T8    FP1-F3    FP2-F4  ...         P3-O1     P4-O2  \
0    -0.000003  0.000140 -0.000018  0.000038  ...  7.37000

In [13]:
pip install scikit-learn --index-url=https://pypi.python.org/simple/

Looking in indexes: https://pypi.python.org/simple/
Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install hmmlearn


   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   ------------------- -------------------- 61.4/125.4 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 125.4/125.4 kB 2.4 MB/s eta 0:00:00


In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from scipy.optimize import curve_fit
import numpy as np
import pandas as pd
import pywt
from hmmlearn import hmm

In [4]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Define wavelet and level of decomposition
wavelet = 'db5'  # Daubechies 5 wavelet, you can choose other wavelets too
level = 6  # Level of decomposition, adjust as needed

# Function to apply DWT to a single channel
def apply_dwt(channel_data):
    coeffs = pywt.wavedec(channel_data, wavelet, level=level)
    return np.concatenate(coeffs)

# Apply DWT to each channel in X
dwt_features = np.apply_along_axis(apply_dwt, axis=1, arr=X)

# Standardize the features
scaler = StandardScaler()
dwt_features_scaled = scaler.fit_transform(dwt_features)

# Perform PCA for compression
n_components = 10  # Number of principal components, adjust as needed
pca = PCA(n_components=n_components)
compressed_features = pca.fit_transform(dwt_features_scaled)

# Save compressed features to a new CSV file
#compressed_data = pd.DataFrame(compressed_features)
#compressed_data['target'] = y  # Add target column
#compressed_data.to_csv('compressed_chbmit_data.csv', index=False)

# Load the compressed CSV file using pandas
trimmed_dataset = pd.read_csv('trimmed_dataset.csv')

# Extract features and labels from the loaded DataFrame
X_features = trimmed_dataset.drop(columns=['target']).values
y_valence_labels = trimmed_dataset['target'].values
y_arousal_labels = trimmed_dataset['target'].values

# Split data into training and testing sets
X_train, X_test, y_train_valence, y_test_valence, y_train_arousal, y_test_arousal = train_test_split(X_features, y_valence_labels, y_arousal_labels, test_size=0.3, random_state=42)

# Ensure there are more than one class in the target variables
if len(np.unique(y_train_valence)) < 2 or len(np.unique(y_train_arousal)) < 2:
    raise ValueError("There must be at least two classes in the target variable for SVM classifiers.")

# Develop Support Vector Machine (SVM) classifiers
# SVM classifier for valence axis
svm_valence = SVC(kernel='rbf')
svm_valence.fit(X_train, y_train_valence)

# SVM classifier for arousal axis
svm_arousal = SVC(kernel='rbf')
svm_arousal.fit(X_train, y_train_arousal)

# Calculate SVM Output
svm_valence_output = svm_valence.decision_function(X_test)
svm_arousal_output = svm_arousal.decision_function(X_test)

# Use Sigmoid Function
def sigmoid(x, A, B):
    return 1 / (1 + np.exp(A * x + B))

def calculate_parameters(X_train, svm_output, y_train):
    # Fit sigmoid function to the SVM output
    params, _ = curve_fit(sigmoid, svm_output, y_train)
    return params

# Calculate Parameters A and B
A_valence, B_valence = calculate_parameters(X_test, svm_valence_output, y_test_valence)
A_arousal, B_arousal = calculate_parameters(X_test, svm_arousal_output, y_test_arousal)

def evaluate_classifier(model, X_test, y_test):
    # Score the model on the test data
    score = accuracy_score(y_test, model.predict(X_test))
    return score

# Combine SVM and Hidden Markov Model (HMM)
# HMM classifier for valence axis
hmm_valence = hmm.GaussianHMM(n_components=2)  # Assuming 2 hidden states
hmm_valence.fit(svm_valence_output.reshape(-1, 1))

# HMM classifier for arousal axis
hmm_arousal = hmm.GaussianHMM(n_components=2)  # Assuming 2 hidden states
hmm_arousal.fit(svm_arousal_output.reshape(-1, 1))

# Evaluate Classifier Performance
valence_accuracy = evaluate_classifier(hmm_valence, X_test, y_test_valence)
arousal_accuracy = evaluate_classifier(hmm_arousal, X_test, y_test_arousal)

print("Valence Accuracy:", valence_accuracy)
print("Arousal Accuracy:", arousal_accuracy)


NameError: name 'df' is not defined